In [1]:
from calcula_resultados import compute_indicators_from_files, load_balancetes_to_df, extract_accounts
import pandas as pd
import json
files = ["balancetes/balancete1.json"]
result = compute_indicators_from_files(files)

In [11]:
# ---- DataFrame por arquivo ----
df_balancete = pd.json_normalize(
    result["per_file"]).drop(columns=["file", "n_leaves"])


df_balancete

,indicators.receita_bruta,indicators.impostos_vendas,indicators.receita_liquida,indicators.custos,indicators.lucro_bruto,indicators.despesas,indicators.lucro_liquido,indicators.disponibilidade_caixa
0,684563.19,3160.22,681402.97,405696.63,275706.34,416239.67,-140533.33,130876.58


In [ ]:
# ==== LEITURA DE UM JSON ====
with open("balancetes/nfeat/Balancete.2023-01.normalized.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
# ==== FUNÇÃO TRANSFORMADORA ====
def extract_accounts(node, hierarchy=None):
    if hierarchy is None:
        hierarchy = []

    current_hierarchy = hierarchy + [node["descricao"]]
    rows = []

    if "children" in node:
        for child in node["children"]:
            rows.extend(extract_accounts(child, current_hierarchy))
    else:
        row = {f"nivel_{i+1}": level for i,
               level in enumerate(current_hierarchy)}
        row["conta"] = node["conta"]
        row["descricao"] = node["descricao"]
        row["saldo_atual"] = node.get("saldo_atual", 0.0)
        rows.append(row)

    return rows

In [4]:
all_rows = []
for section in data.values():
    if isinstance(section, dict) and "descricao" in section:
        all_rows.extend(extract_accounts(section))

# Create a DataFrame
df = pd.DataFrame(all_rows)

In [ ]:
# ==== LÊ TODOS OS BALANCETES ====
import os
import json
import pandas as pd

# Caminho da pasta
folder_path = "balancetes/industrial_nordeste"

all_rows = []

# Loop em todos os arquivos JSON da pasta
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)

        # Extrai o mês do nome do arquivo (ex: Balancete.2023-01.normalized.json → 2023-01)
        mes = filename.split(".")[1]

        # Lê o JSON
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        # Percorre as seções e extrai as contas
        for section in data.values():
            if isinstance(section, dict) and "descricao" in section:
                contas = extract_accounts(section)
                # Adiciona o mês a cada conta extraída
                for conta in contas:
                    conta["mes"] = mes
                all_rows.extend(contas)

# Cria o DataFrame consolidado
df = pd.DataFrame(all_rows)



In [37]:
df

,nivel_1,nivel_2,nivel_3,nivel_4,nivel_5,conta,descricao,saldo_atual,mes
0,ATIVO,ATIVO CIRCULANTE,DISPONIBILIDADES,CAIXA,Caixa,01.1.1.01.001,Caixa,109463.73,2023-01
1,ATIVO,ATIVO CIRCULANTE,DIREITOS REALIZÁVEIS A CURTO PRAZO,CLIENTES NACIONAIS,Clientes nacionais,01.1.2.01.001,Clientes nacionais,34968.86,2023-01
2,ATIVO,ATIVO CIRCULANTE,DIREITOS REALIZÁVEIS A CURTO PRAZO,TRIBUTOS E CONTRIBUIÇÕES A COMPENSAR,ICMS a Recuperar,01.1.2.08.001,ICMS a Recuperar,1252.64,2023-01
3,ATIVO,ATIVO CIRCULANTE,DIREITOS REALIZÁVEIS A CURTO PRAZO,ESTOQUES,Estoque de Mercadoria Para Revenda,01.1.2.10.001,Estoque de Mercadoria Para Revenda,12874.09,2023-01
4,ATIVO,ATIVO CIRCULANTE,DIREITOS REALIZÁVEIS A CURTO PRAZO,REMESSA DE PRODUTOS/MERCADORIAS,Remessa de Produtos/Mercadorias,01.1.2.12.001,Remessa de Produtos/Mercadorias,13287.53,2023-01
...,...,...,...,...,...,...,...,...,...
367,CUSTOS E DESPESAS,Depreciações e Amortizações,NaN,NaN,NaN,04.2.1.99.013,Depreciações e Amortizações,167.23,2023-12
368,CUSTOS E DESPESAS,Fretes e Carretos,NaN,NaN,NaN,04.2.1.99.015,Fretes e Carretos,628.73,2023-12
369,CUSTOS E DESPESAS,Serviços de Terceiros - Pessoa Jurídica,NaN,NaN,NaN,04.2.1.99.020,Serviços de Terceiros - Pessoa Jurídica,315550.60,2023-12
370,CUSTOS E DESPESAS,Juros,NaN,NaN,NaN,04.2.2.02.001,Juros,119.42,2023-12


In [40]:
receita_bruta = df[(df["nivel_1"] == "RECEITAS") & (
    df["nivel_2"] == "Serviços Prestados a Prazo")]

# Impostos sobre receita (deduções) = "Simples Nacional sobre vendas e serviços"
impostos_sobre_receita = df[(df["nivel_1"] == "RECEITAS") & (
    df["nivel_2"] == "Simples Nacional sobre vendas e serviços")]

# Custos e Despesas = todas as linhas em "CUSTOS E DESPESAS"
custos_despesas = df[df["nivel_1"] == "CUSTOS E DESPESAS"]

# Caixa e equivalentes = "ATIVO CIRCULANTE" e "DISPONIBILIDADES"
caixa = df[(df["nivel_2"] == "ATIVO CIRCULANTE") &
           (df["nivel_3"] == "DISPONIBILIDADES")]

# Agregar valores por mês
receita_bruta_mensal = receita_bruta.groupby("mes")["saldo_atual"].sum()
impostos_mensal = impostos_sobre_receita.groupby("mes")["saldo_atual"].sum()
custos_mensal = custos_despesas.groupby("mes")["saldo_atual"].sum()
caixa_mensal = caixa.groupby("mes")["saldo_atual"].sum()

# Calcular os indicadores
df_indices = pd.DataFrame({
    "Receita Bruta": receita_bruta_mensal,
    "(-) Impostos sobre Receita": impostos_mensal,
    "Custo Total": custos_mensal,
    "Disponibilidade de Caixa": caixa_mensal
})

# Receita Líquida
df_indices["Receita Líquida"] = df_indices["Receita Bruta"] - \
    df_indices["(-) Impostos sobre Receita"]

# Lucro Bruto = Receita Líquida - Custo Total
df_indices["Lucro Bruto"] = df_indices["Receita Líquida"] - \
    df_indices["Custo Total"]

# Lucro Líquido (neste modelo simplificado consideramos que todos os custos/despesas já estão no grupo "CUSTOS E DESPESAS")
df_indices["Lucro Líquido"] = df_indices["Lucro Bruto"]

In [41]:
df_indices

,Receita Bruta,(-) Impostos sobre Receita,Custo Total,Disponibilidade de Caixa,Receita Líquida,Lucro Bruto,Lucro Líquido
mes,,,,,,,
2023-01,429901.40,-37005.85,394821.21,109463.73,466907.25,72086.04,72086.04
2023-02,321346.85,-36053.27,421294.61,117390.16,357400.12,-63894.49,-63894.49
2023-03,440831.99,-39032.98,322318.55,118591.10,479864.97,157546.42,157546.42
2023-04,327009.19,-46053.09,398476.89,108421.05,373062.28,-25414.61,-25414.61
2023-05,418292.76,-30113.69,308838.38,100677.15,448406.45,139568.07,139568.07
2023-06,409987.78,-32296.83,363654.46,109264.96,442284.61,78630.15,78630.15
2023-07,426368.02,-42460.42,419743.75,117555.50,468828.44,49084.69,49084.69
2023-08,341142.69,-39765.13,406375.28,87176.40,380907.82,-25467.46,-25467.46
2023-09,396731.67,-39909.61,350382.84,125104.32,436641.28,86258.44,86258.44


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   nivel_1      372 non-null    object 
 1   nivel_2      372 non-null    object 
 2   nivel_3      276 non-null    object 
 3   nivel_4      240 non-null    object 
 4   nivel_5      84 non-null     object 
 5   conta        372 non-null    object 
 6   descricao    372 non-null    object 
 7   saldo_atual  372 non-null    float64
 8   mes          372 non-null    object 
dtypes: float64(1), object(8)
memory usage: 26.3+ KB


In [ ]:

df[df["nivel_1"].str.startswith("A")]

,nivel_1,nivel_2,nivel_3,nivel_4,conta,descricao,saldo_atual


In [59]:
df.drop_duplicates(subset=['nivel_2', 'nivel_3'])

,nivel_1,nivel_2,nivel_3,nivel_4,nivel_5,conta,descricao,saldo_atual,mes
0,ATIVO,ATIVO CIRCULANTE,DISPONIBILIDADES,CAIXA,Caixa,01.1.1.01.001,Caixa,109463.73,2023-01
1,ATIVO,ATIVO CIRCULANTE,DIREITOS REALIZÁVEIS A CURTO PRAZO,CLIENTES NACIONAIS,Clientes nacionais,01.1.2.01.001,Clientes nacionais,34968.86,2023-01
5,ATIVO,ATIVO NÃO CIRCULANTE,IMOBILIZADO,BENS E DIREITOS EM USO,Máquinas e Equipamentos,01.2.3.01.001,Máquinas e Equipamentos,20355.33,2023-01
7,PASSIVO,PASSIVO CIRCULANTE,FORNECEDORES,REZENDE CENTRO COMERCIAL AUTOMOTIVO LTDA,NaN,02.1.1.01.000007895,REZENDE CENTRO COMERCIAL AUTOMOTIVO LTDA,2106.89,2023-01
12,PASSIVO,PASSIVO CIRCULANTE,EMPRÉSTIMOS E FINANCIAMENTOS,Banco Bradesco S/A,NaN,02.1.2.01.010,Banco Bradesco S/A,23552.35,2023-01
13,PASSIVO,PASSIVO CIRCULANTE,OBRIGAÇÕES TRABALHISTAS,Salários a pagar,NaN,02.1.3.01.001,Salários a pagar,43593.85,2023-01
15,PASSIVO,PASSIVO CIRCULANTE,OBRIGAÇÕES TRIBUTÁRIAS,IRRF a Recolher,NaN,02.1.4.01.001,IRRF a Recolher,557.99,2023-01
20,PASSIVO,PASSIVO NÃO CIRCULANTE,Bens Remetidos para Teste,NaN,NaN,02.2.1.02.002,Bens Remetidos para Teste,14602.65,2023-01
21,PASSIVO,PATRIMÔNIO LÍQUIDO,Capital Social Integralizado,NaN,NaN,02.3.1.01.002,Capital Social Integralizado,8740.59,2023-01
22,PASSIVO,PATRIMÔNIO LÍQUIDO,Reserva de Lucros,NaN,NaN,02.3.2.02.001,Reserva de Lucros,71143.86,2023-01


In [62]:
df_agrupado = df.groupby(['nivel_2', 'mes'], as_index=False)['saldo_atual'].sum()
filtro = df_agrupado.loc[
    df_agrupado['nivel_2'].isin(
        ['ATIVO CIRCULANTE', 'PASSIVO CIRCULANTE', 'ATIVO NÃO CIRCULANTE', 'PASSIVO NÃO CIRCULANTE'])
]
# Cria a tabela pivot
tabela_pivot = pd.pivot_table(
    filtro,
    values='saldo_atual',
    index='mes',
    columns='nivel_2',
    aggfunc='first'
)

print("Tabela original:")
df_agrupado
print("\nTabela pivotada:")
tabela_pivot

# Criar os cálculos
tabela_pivot['Ativo_Total'] = tabela_pivot['ATIVO CIRCULANTE'] + tabela_pivot['ATIVO NÃO CIRCULANTE']
tabela_pivot['Passivo_Total'] = tabela_pivot['PASSIVO CIRCULANTE'] + tabela_pivot['PASSIVO NÃO CIRCULANTE']

tabela_pivot['Liquidez_Corrente'] = tabela_pivot['ATIVO CIRCULANTE'] / tabela_pivot['PASSIVO CIRCULANTE']
# tabela_pivot['Liquidez_Imediata'] = tabela_pivot['Disponibilidades'] / tabela_pivot['PASSIVO CIRCULANTE']
tabela_pivot['Liquidez_Geral'] = (tabela_pivot['ATIVO CIRCULANTE'] + tabela_pivot['ATIVO NÃO CIRCULANTE']) / \
    (tabela_pivot['PASSIVO CIRCULANTE'] + tabela_pivot['PASSIVO NÃO CIRCULANTE'])
tabela_pivot['Solvencia_Geral'] = tabela_pivot['Ativo_Total'] / tabela_pivot['Passivo_Total']
tabela_pivot['Endividamento'] = tabela_pivot['Passivo_Total'] / tabela_pivot['Ativo_Total']
tabela_pivot['Endividamento_Geral'] = (tabela_pivot['Passivo_Total']) / tabela_pivot['Ativo_Total']

Tabela original:

Tabela pivotada:


In [64]:
tabela_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 2023-01 to 2023-12
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ATIVO CIRCULANTE        12 non-null     float64
 1   ATIVO NÃO CIRCULANTE    12 non-null     float64
 2   PASSIVO CIRCULANTE      12 non-null     float64
 3   PASSIVO NÃO CIRCULANTE  12 non-null     float64
 4   Ativo_Total             12 non-null     float64
 5   Passivo_Total           12 non-null     float64
 6   Liquidez_Corrente       12 non-null     float64
 7   Liquidez_Geral          12 non-null     float64
 8   Solvencia_Geral         12 non-null     float64
 9   Endividamento           12 non-null     float64
 10  Endividamento_Geral     12 non-null     float64
dtypes: float64(11)
memory usage: 1.4+ KB


In [47]:
df_agrupado.head(50)

,nivel_2,mes,saldo_atual
0,ATIVO CIRCULANTE,2023-01,171846.85
1,ATIVO CIRCULANTE,2023-02,194311.17
2,ATIVO CIRCULANTE,2023-03,195146.77
3,ATIVO CIRCULANTE,2023-04,180656.01
4,ATIVO CIRCULANTE,2023-05,169029.08
5,ATIVO CIRCULANTE,2023-06,176836.56
6,ATIVO CIRCULANTE,2023-07,177204.61
7,ATIVO CIRCULANTE,2023-08,158219.08
8,ATIVO CIRCULANTE,2023-09,186976.07
9,ATIVO CIRCULANTE,2023-10,178534.83


In [15]:
import json
import random


def modificar_valores_json(json_data):
    def modificar_nodo(nodo):
        # Se o nó tiver filhos, processa-os recursivamente
        if 'children' in nodo:
            for filho in nodo['children']:
                modificar_nodo(filho)
            return

        # Modifica os valores financeiros com variação de ±25%
        for campo in ['saldo_anterior', 'debito', 'credito']:
            if campo in nodo:
                valor_original = nodo[campo]
                # Gera uma variação aleatória entre -25% e +25%
                variacao = random.uniform(-0.25, 0.25)
                novo_valor = valor_original * (1 + variacao)
                nodo[campo] = round(novo_valor, 2)

        # Calcula o novo saldo_atual baseado no tipo de conta
        if 'conta' in nodo:
            conta = nodo['conta']
            if conta.startswith(('01', '04')):  # ATIVO e CUSTOS
                nodo['saldo_atual'] = round(nodo.get('saldo_anterior', 0) +
                                            nodo.get('debito', 0) -
                                            nodo.get('credito', 0), 2)
            elif conta.startswith(('02', '03')):  # PASSIVO e RECEITAS
                nodo['saldo_atual'] = round(nodo.get('saldo_anterior', 0) -
                                            nodo.get('debito', 0) +
                                            nodo.get('credito', 0), 2)

    # Processa cada seção do JSON
    for secao in json_data.values():
        if isinstance(secao, dict) and 'conta' in secao:
            modificar_nodo(secao)

    return json_data

In [35]:
# ==== LEITURA DE UM JSON ====
with open("balancetes/nfeat/Balancete.2023-12.normalized.json", "r", encoding="utf-8") as f:
    json_original = json.load(f)

json_modificado = modificar_valores_json(json_original)

# Salva o arquivo
with open('balancetes/industrial_nordeste/Balancete.2023-12.json', 'w', encoding='utf-8') as arquivo:
    json.dump(json_modificado, arquivo, indent=4)